In [6]:
import json
import pandas as pd
import re
from google.cloud import bigquery

In [19]:
with open("output_wolt_finland.json", mode="r", encoding="utf-8") as f:
    data = json.load(f)
    df_data = pd.DataFrame(data)
    f.close()

In [20]:
df_data["delivery_time_min"] = df_data["delivery_time"].apply(lambda x: int(x.split("-")[0]))
df_data["delivery_time_max"] = df_data["delivery_time"].apply(lambda x: int(x.split("-")[1]))
df_data["delivery_fee_numeric"] = df_data["delivery_fee"].apply(lambda x: float('.'.join(re.findall(pattern="\d+", string=x))))
df_data["scraping_timestamp"] = df_data["scraping_timestamp"].apply(lambda x: pd.to_datetime(x))

In [21]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   restaurant_name       90 non-null     object        
 1   restaurant_subtitle   90 non-null     object        
 2   delivery_time         90 non-null     object        
 3   delivery_fee          90 non-null     object        
 4   scraping_timestamp    90 non-null     datetime64[ns]
 5   scraping_location     90 non-null     object        
 6   scraping_country      90 non-null     object        
 7   delivery_time_min     90 non-null     int64         
 8   delivery_time_max     90 non-null     int64         
 9   delivery_fee_numeric  90 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 7.2+ KB


In [22]:
# Upload the data to bigquery
client = bigquery.Client(project="dh-logistics-product-ops")

job_config = bigquery.LoadJobConfig(
    schema = [
        bigquery.SchemaField("restaurant_name", "STRING"),
        bigquery.SchemaField("restaurant_subtitle", "STRING"),
        bigquery.SchemaField("delivery_time", "STRING"),
        bigquery.SchemaField("delivery_fee", "STRING"),
        bigquery.SchemaField("scraping_timestamp", "TIMESTAMP"),
        bigquery.SchemaField("scraping_location", "STRING"),
        bigquery.SchemaField("scraping_country", "STRING"),
        bigquery.SchemaField("delivery_time_min", "INT64"),
        bigquery.SchemaField("delivery_time_max", "INT64"),
        bigquery.SchemaField("delivery_fee_numeric", "FLOAT64"),
    ]
)
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

client.load_table_from_dataframe(
    dataframe=df_data,
    destination="dh-logistics-product-ops.pricing.wolt_finland_crawler",
    job_config=job_config
)

i:\My Drive\AUC Drive\Entrepreneurial Path and Financial Freedom\Freelancing\scraping_gigs\venv_scraping\lib\site-packages\google\auth\_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


LoadJob<project=dh-logistics-product-ops, location=US, id=cd8e1ca2-fc98-48a8-bcaf-50ac395a52db>